# AI21 Maestro — Usage Example

Here you will find simple usage example for AI21 Maestro. For more information, check out ai21 [technical documentation](https://docs.ai21.com/).

**Setup.** Import Maestro SDK classes. Make sure your API key is configured (env or client init). You can find your api key [here](https://studio.ai21.com/v2/account/api-key?source=docs&_gl=1*u1mkxz*_gcl_au*OTIzMDQ5NzU0LjE3NDkyOTE4NjA).

In [ ]:
# Setup and initialization
!pip install ai21

**Setup.** Import Maestro SDK classes. Make sure your API key is configured (env or client init).

In [1]:
from ai21 import AsyncAI21Client

# Get your API key from https://studio.ai21.com
AI21_API_KEY = "YOUR_API_KEY"
client = AsyncAI21Client(api_key=AI21_API_KEY)

## Generate with Requirements
Use requirements to get more accurate results. Maestro will validate that all your requirements are met, and will provide a report card at the end of the run.

The parameters:

1. `requirements` is a list of either:
   - `Requirement` objects
   - an ordinary dictionary with the keys `name` and `description`.
2. Use `include` to get a `requirements_result` with a full report of which requirements were actually fulfilled.
3. `models` is a list of strings representing Model integration names you previously set in [AI21 Studio](https://studio.ai21.com/).

Let's look at the example:
We want to extract information from an unstructured text, but without the LLM making up things.

First, we will build the prompt and define the requirements:

In [3]:
prompt = """You are an expert in extracting structured information from unstructured text, specifically travel booking conversations.
You are given a conversation as plain text between a user and an AI travel agent. Extract and return the following fields:
Name: Full name of the traveler
HotelName: Name of the hotel selected by the traveler
CheckInDate: Check-in date in YYYY-MM-DD format if the date can fully be extracted
CheckOutDate: Check-out date in YYYY-MM-DD format if the date can fully be extracted
City: City where the hotel is located
Country: Country where the hotel is located
NumberOfGuests: Total number of guests"""

conversation = """Conversation:
User: Hi there! I'm planning a trip to Rome next month and need help booking a hotel.
AI: I'd be happy to help! Could you tell me your travel dates and how many people will be staying?
User: Sure. I'll be arriving on July 10th and checking out on July 15th. It'll be just me and my wife, so 2 guests.
AI: Great. Would you like me to recommend some hotels in Rome, Italy?
User: Yes, please.
AI: Here are a few options:
Hotel Artemide
The Rome EDITION
Hotel Nazionale
Do any of these interest you?
User: Hotel Artemide sounds perfect. Let's go with that.
AI: Excellent choice! May I have your full name for the reservation?
User: It's Jonathan Rivera.
AI: Thank you, Jonathan. I've reserved a room at Hotel Artemide in Rome, Italy for 2 guests from July 10 to July 15."""

In [4]:
import json

run_input = f"""{prompt}

{conversation}"""

run = await client.beta.maestro.runs.create_and_poll(
    input=run_input,
    requirements=[
        Requirement(
            name="bullets",
            description="Return a bulleted list where each field is a bullet."
        ),
        Requirement(
            name="NA",
            description='If any field is not mentioned or can’t be fully extracted from the conversation, fill it with "NA".'
        ),
        Requirement(
            name="don't make things up",
            description="Do not invent, infer, or assume any details that are not explicitly stated in the conversation."
        ),
    ],
    include=["requirements_result"],
    budget="low",
)

Let's take a look at the output and the report card on the requirements:

In [5]:
print(f"Run {run.id} status: {run.status}.\nResult:")
print(run.result)
print("\n************************\n")
print("Requirements result:")
print(json.dumps(run.requirements_result, indent=2))

Run 068cdf15-48e4-7757-8000-cc73d015d060 status: completed.
Result:
- Name: Jonathan Rivera
- HotelName: Hotel Artemide
- CheckInDate: 2023-07-10
- CheckOutDate: 2023-07-15
- City: Rome
- Country: Italy
- NumberOfGuests: 2

************************

Requirements result:
{
  "score": 0.9,
  "finish_reason": "Budget exhausted",
  "requirements": [
    {
      "name": "bullets",
      "description": "Return a bulleted list where each field is a bullet.",
      "is_mandatory": false,
      "score": 1.0,
      "reason": "None"
    },
    {
      "name": "NA",
      "description": "If any field is not mentioned or can\u2019t be fully extracted from the conversation, fill it with \"NA\".",
      "is_mandatory": false,
      "score": 1.0,
      "reason": "None"
    },
    {
      "name": "don't make things up",
      "description": "Do not invent, infer, or assume any details that are not explicitly stated in the conversation.",
      "is_mandatory": false,
      "score": 0.6,
      "reason": 

# External Tools:


## Http
For now, Maestro supports only POST requests.

We will show a simple example of using Echo API.
First, we will define the http tool:

In [6]:
import json
from ai21.models.maestro import HttpTool, Function, Parameters, Endpoint

echo_tool = HttpTool(
    type="http",
    function=Function(
        name="echo_post_httpbin",
        description="Echo back a message string using httpbin.org",
        parameters=Parameters(
            type="object",
            properties={
                "message": {
                    "type": "string",
                    "description": "Any string to echo back"
                }
            },
            required=["message"],
        ),
    ),
    endpoint=Endpoint(
        url="https://httpbin.org/post"
    ),
)

Now, we will pass that tool to Maestro.

In [7]:
message = "Stop repeating everything I say!"

run_result = await client.beta.maestro.runs.create_and_poll(
    input=(
        f"Call the tool 'echo_post_httpbin' with:\n"
        f'{{"message": "{message}"}}\n'
        "Return the tool's raw JSON response."
    ),
    tools=[echo_tool],
)

# httpbin puts echoed content under 'json'
resp = json.loads(run_result.result)

In [8]:
print("Original message:", message)
print("Agent answer:", resp.get("json", {}).get("message"))

Original message: Stop repeating everything I say!
Agent answer: Stop repeating everything I say!


## MCP
You can use an existing MCP or connect your own.
If you use your own, use ngrok or a similar tool to expose your local MCP server to the internet. [Full documentation](https://docs.ai21.com/docs/mcp-server-setup-guide).

In [3]:
from ai21.models.maestro import McpTool
from IPython.display import Markdown, display

run_base_result = await client.beta.maestro.runs.create_and_poll(
    input="What's the weather like in San Fransisco?",
    budget="low",
    tools=[
        McpTool(
            type="mcp",
            server_url="https://jiri-spilka--weather-mcp-server.apify.actor/mcp",
            server_label="weather-mcp",
            headers={"Authorization": "Bearer YOUR_API_TOKEN"},
        )]
)

display(Markdown(run_base_result.result))

The weather in San Francisco is partly cloudy with a temperature of 18.3°C, relative humidity at 2 meters is 82%, and the dew point temperature at 2 meters is 15.2°C.

# Enable File Search (RAG)

You can upload files to the library and use "file_search" as a tool to Maestro.
In this example, we will upload the [following page from Wikipedia](https://en.wikipedia.org/wiki/List_of_Pok%C3%A9mon) (downloaded as pdf).

In [9]:
file_path = "data/List_of_Pokémon.pdf"

file_id = await client.library.files.create(
    file_path=str(file_path)
)

print(file_id)

a9fa288d-a6b6-4a54-b294-f09b14978cbe


Now enable the `file_search` tool in Maestro:

In [5]:
run_result = await client.beta.maestro.runs.create_and_poll(
    input=(
        "Give me a list of 5 pokemons from the first generation and 4 pokemons from generation 3"
    ),
    tools=[{"type": "file_search"}],
    include=["data_sources"],
)

Let's see the result:

In [6]:
from IPython.display import Markdown, display

display(Markdown(run_result.result))

Here is a list of 5 Pokémon from the first generation and 4 Pokémon from the third generation based on the provided context:

**Generation 1 Pokémon:**
1. Bulbasaur
2. Ivysaur
3. Venusaur
4. Charmander
5. Charmeleon

**Generation 3 Pokémon:**
1. Treecko
2. Grovyle
3. Sceptile
4. Torchic

In [10]:
import json
print(json.dumps(run_result.data_sources, indent=2))


{
  "file_search": [
    {
      "text": "** | **New in generation** | **Total** |\n| I | 1996\u20131999 | Kanto | Red, Green, Blue, and Yellow | Game Boy, Nintendo 3DS[b] | 151 | 151 | 151 |\n| II | 1999\u20132002 | Johto, Kanto | Gold, Silver, and Crystal | Game Boy Color, Nintendo 3DS[b] | 100 | 100 | 251 |\n| III | 2002\u20132006 | Hoenn | Ruby, Sapphire, and Emerald | Game Boy Advance | 135 | 135 | 386 |\n| III | 2002\u20132006 | Kanto | FireRed and LeafGreen | Game Boy Advance | None | 135 | 386 |\n| IV | 2006\u20132010 | Sinnoh | Diamond, Pearl, and Platinum | Nintendo DS | 107 | 107 | 493 |\n| IV | 2006\u20132010 | Johto, Kanto | HeartGold and SoulSilver | Nintendo DS | None | 107 | 493 |\n| V | 2010\u20132013 | Unova | Black and White | Nintendo DS | 156 | 156 | 649 |\n| V | 2010\u20132013 | Unova | Black 2 and White 2 | Nintendo DS | None | 156 | 649 |\n| VI | 2013\u20132016 | Kalos | X and Y | Nintendo 3DS | 72 | 72 | 721 |\n| VI | 2013\u20132016 | Hoenn | Omega Ruby and Alp

# Web Search
You can use web search as a tool to Maestro.

In [13]:
run_result = await client.beta.maestro.runs.create_and_poll(
    input=(
        "Give me a list of 5 pokemons from the first generation and 4 pokemons from generation 3"
    ),
    tools=[{"type": "web_search"}],
    include=["data_sources"],
)
display(Markdown(run_result.result))


Here are 5 Pokémon from the first generation:
1. Bulbasaur
2. Charmander
3. Squirtle
4. Pikachu
5. Jigglypuff

And here are 4 Pokémon from the third generation:
1. Treecko
2. Torchic
3. Mudkip
4. Zigzagoon

# Chat messages (history)
`input` can be either a `string` or a list of dictionaries, consists of `role` and `content`.

In [12]:
run = await client.beta.maestro.runs.create_and_poll(
    input=[
        {"role": "user", "content": "hi"},
        {"role": "assistant", "content": "Hello, how may I assist you today?"},
        {"role": "user", "content": "Which Pokemon is the best?"},
    ],
    budget="low",
    poll_interval_sec=1,
    poll_timeout_sec=20,
)

print("Result:", run.result)

Result: Arceus
